In [1]:
import pandas as pd, numpy as np, re, random, os
import matplotlib.pyplot as plt, seaborn as sns
from scipy import stats
from itertools import permutations, cycle, combinations
from glob import glob

In [2]:
def generate_trial_list(nTrials):
    trial_list = [0] * nTrials  # Initialize the list with zeros
    consecutive_zeros = 0

    while sum(trial_list) != 6:
        for i in range(nTrials):
            if i == 0:
                trial_list[i] = 0
                consecutive_zeros += 1
            elif i == 1:
                trial_list[i] = 0
                consecutive_zeros += 1
            elif i == nTrials-2:
                trial_list[i] = 0
                consecutive_zeros += 1
            elif i == nTrials-1:
                trial_list[i] = 1
                consecutive_zeros += 1
            else:
                if consecutive_zeros >= random.randint(1, 3):
                    # Ensure that a random number of consecutive zeros (3-4) have passed
                    trial_list[i] = 1
                    consecutive_zeros = 0
                else:
                    trial_list[i] = 0
                    consecutive_zeros += 1

    return trial_list

nTrials = 16  # You can set the desired number of trials
result_list = generate_trial_list(nTrials)
print(result_list)

[0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1]


In [3]:
fix_selected_photos =pd.read_csv('C:/Users/Shawn/Documents/GitHub/task_filmreels/images/selected_photos.tsv', sep='\t')

In [4]:
reel_combos = {'A':[['101','306'],
                    ['202','107'],
                    ['304','208'],
                    ['203','305']],
               'B':[['206','307'],
                    ['301','105'],
                    ['108','303'],
                    ['102','204']],
               'C':[['201','302'],
                    ['103','207'],
                    ['104','205'],
                    ['308','106']]}

In [6]:
all_orders = permutations(['social', 'nature', 'food'])
cyclic_orders = cycle(all_orders)

reel_orders = permutations(['A','B','C'])
cyclic_reel_orders = cycle(reel_orders)

ncount = 0
prob_of_cue = 0.75
nconds=1
possible_trial_type = 'n-016_cond-1_block-1_trial-16'  # 48 trials/condition

for subject_id in [9999]+list(range(5114,5476)):
    # split each category into 4 block each and randomize the order of the 4 blocks
    np.random.seed(subject_id+500)
    social_photos = []
    nature_photos = []
    food_photos = []
    tmp_social_photos = fix_selected_photos[fix_selected_photos['category'] == 'social'].sample(n=64)
    tmp_nature_photos = fix_selected_photos[fix_selected_photos['category'] == 'nature'].sample(n=64)
    tmp_food_photos = fix_selected_photos[fix_selected_photos['category'] == 'food'].sample(n=64)
    for i in range(4):
        # cut each into 4 equal chunks
        social_photos.append(tmp_social_photos[i*16:(i+1)*16])
        nature_photos.append(tmp_nature_photos[i*16:(i+1)*16])
        food_photos.append(tmp_food_photos[i*16:(i+1)*16])

    # get stim list
    this_reel_order = list(next(cyclic_reel_orders))
    perm_order = np.random.permutation([0,1,2,3])
    lr_order = np.random.permutation([0,1])

    stimList1 = np.array([[reel_combos[this_reel_order[0]][perm_order[0]][lr_order[0]], reel_combos[this_reel_order[0]][perm_order[0]][lr_order[1]]],
                          [reel_combos[this_reel_order[0]][perm_order[1]][lr_order[0]], reel_combos[this_reel_order[0]][perm_order[1]][lr_order[1]]],
                          [reel_combos[this_reel_order[0]][perm_order[2]][lr_order[0]], reel_combos[this_reel_order[0]][perm_order[2]][lr_order[1]]],
                          [reel_combos[this_reel_order[0]][perm_order[3]][lr_order[0]], reel_combos[this_reel_order[0]][perm_order[3]][lr_order[1]]]])
    stimList2 = np.array([[reel_combos[this_reel_order[1]][perm_order[0]][lr_order[0]], reel_combos[this_reel_order[1]][perm_order[0]][lr_order[1]]],
                          [reel_combos[this_reel_order[1]][perm_order[1]][lr_order[0]], reel_combos[this_reel_order[1]][perm_order[1]][lr_order[1]]],
                          [reel_combos[this_reel_order[1]][perm_order[2]][lr_order[0]], reel_combos[this_reel_order[1]][perm_order[2]][lr_order[1]]],
                          [reel_combos[this_reel_order[1]][perm_order[3]][lr_order[0]], reel_combos[this_reel_order[1]][perm_order[3]][lr_order[1]]]])
    stimList3 = np.array([[reel_combos[this_reel_order[2]][perm_order[0]][lr_order[0]], reel_combos[this_reel_order[2]][perm_order[0]][lr_order[1]]],
                          [reel_combos[this_reel_order[2]][perm_order[1]][lr_order[0]], reel_combos[this_reel_order[2]][perm_order[1]][lr_order[1]]],
                          [reel_combos[this_reel_order[2]][perm_order[2]][lr_order[0]], reel_combos[this_reel_order[2]][perm_order[2]][lr_order[1]]],
                          [reel_combos[this_reel_order[2]][perm_order[3]][lr_order[0]], reel_combos[this_reel_order[2]][perm_order[3]][lr_order[1]]]])

    stim_list = np.concatenate((stimList1, stimList2, stimList3), axis=0)
    
    # condition_list = ['social','nature','food','food','social','nature','nature','food','social','social','food','nature']
    # # while any adjacent items in the list are the same, shuffle
    # while any([condition_list[i] == condition_list[i+1] for i in range(len(condition_list)-1)]):
    #     tmp_list = []
    #     for i in range(4):        
    #         tmp_list += [x for x in np.random.permutation(['social','nature','food'])]
    #     condition_list = tmp_list
    condition_list = list(next(cyclic_orders))
    condition_list = [condition_list[0]] * 4 + [condition_list[1]] * 4 + [condition_list[2]] * 4

    block_index = 0
    count_social = 0
    count_nature = 0
    count_food = 0
    for condition in condition_list:
        lonely_rate = []
        match = re.match(r'n-(\d+)_cond-\d+_block-(\d+)_trial-(\d+)', possible_trial_type)
        if match:
            total_trials = int(match.group(1))
            nblocks      = int(match.group(2))
            ntrials      = int(match.group(3))
        else:
            total_trials = np.nan
            nblocks      = np.nan
            ntrials      = np.nan

        # get trial structure for block
        trial_positions = pd.read_excel(f'social-reels_trial-template.xlsx',
                                        sheet_name=possible_trial_type,
                                        engine='openpyxl')
        this_block_df = trial_positions.copy()

        # get photos for block
        if condition == 'social':
            this_block_images = social_photos[count_social].sample(frac=1)
            count_social += 1
        elif condition == 'nature':
            this_block_images = nature_photos[count_nature].sample(frac=1)
            count_nature += 1
        elif condition == 'food':
            this_block_images = food_photos[count_food].sample(frac=1)
            count_food += 1
        this_block_images.index = range(len(this_block_images))

        # set cues based on probability per block
        block_outcome_template = [1]*int(prob_of_cue*ntrials) + [0]*int((1-prob_of_cue)*ntrials)
        
        # generate trial structure (without outcomes repeating three times in a row)
        check_threepeats = 1
        count = 0
        while check_threepeats != 0:
            # random state changes in while loop, also multiplied by the magnitude of run number
            np.random.seed((block_index+subject_id)+(ncount+count))
            random_index = np.random.permutation(ntrials)
            this_block_df['option_left'] = list(this_block_df['option_left'].iloc[random_index])
            this_block_df['option_right'] = list(this_block_df['option_right'].iloc[random_index])

            # shuffle rows in df
            # this_block_df = this_block_df.sample(frac=1, random_state=block_index*(subject_id*ncount))

            # count number of 
            number_consecutive_repeats = list(this_block_df.groupby(this_block_df['option_left'].ne(this_block_df['option_left'].shift()).cumsum())['option_left'].transform('size')) 

            # check if any are repeated more than 3 times in row
            check_threepeats = sum([1 if x > 3 else 0 for x in number_consecutive_repeats])

            count += 1

        # set stim
        for idx, row in this_block_df.iterrows():
            if row['option_left'] == 'A':
                this_block_df.at[idx, 'stim_left'] = 'stimuli/filmreel_'+stim_list[block_index,0]+'.png'
                this_block_df.at[idx, 'stim_right'] = 'stimuli/filmreel_'+stim_list[block_index,1]+'.png'
            elif row['option_left'] == 'B':
                this_block_df.at[idx, 'stim_left'] = 'stimuli/filmreel_'+stim_list[block_index,1]+'.png'
                this_block_df.at[idx, 'stim_right'] = 'stimuli/filmreel_'+stim_list[block_index,0]+'.png'

        # randomize order of outcomes
        np.random.shuffle(block_outcome_template)
        this_block_df['A_outcome'] = list(block_outcome_template)
        this_block_df['B_outcome'] = [1 if r==0 else 0 for r in this_block_df['A_outcome']]

        # jitter craving ratings
        craving_rate = generate_trial_list(ntrials)
        this_block_df['rate_craving'] = craving_rate

        # merge this_block_df with photo df
        this_block_df = pd.merge(this_block_df, this_block_images, left_index=True, right_index=True)
        
        # add condition
        this_block_df['condition'] = condition
        this_block_df['blockIndex'] = block_index

        # append block_df
        if block_index==0:
            subj_df = this_block_df.copy()
        else:
            subj_df = subj_df.append(this_block_df, ignore_index=True)

        block_index += 1
        ncount += 1

    subj_df.drop(columns='embedding', inplace=True)
    subj_df.rename(columns={'index':'photoIndex'}, inplace=True)
    subj_df.index.name = 'index'
    
    if subject_id == 9999:
        # only use block_index 0,4,8
        subj_df = subj_df[subj_df['blockIndex'].isin([0,4,8])]
        
    subj_df.to_csv(f'sub-{subject_id}_task-filmreels_orders.csv', index=True)

C:\Users\Shawn\AppData\Local\Temp\ipykernel_25384\1205161484.py:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subj_df = subj_df.append(this_block_df, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_25384\1205161484.py:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subj_df = subj_df.append(this_block_df, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_25384\1205161484.py:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subj_df = subj_df.append(this_block_df, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_25384\1205161484.py:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subj_df